In [1]:
import pandas as pd

In [2]:
data_dir = '../data'

In [3]:
df = pd.read_csv(f"{data_dir}/raw/train.csv", index_col='Id').convert_dtypes()
# df = pd.read_csv(f"{data_dir}/raw/test.csv", index_col='Id').convert_dtypes()
display(df.shape)
df.head(2)

(1460, 80)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65,8450,Pave,<NA>,Reg,Lvl,AllPub,Inside,...,0,<NA>,<NA>,<NA>,0,2,2008,WD,Normal,208500
2,20,RL,80,9600,Pave,<NA>,Reg,Lvl,AllPub,FR2,...,0,<NA>,<NA>,<NA>,0,5,2007,WD,Normal,181500


In [4]:
cols_fillna_zero = [
    "GarageYrBlt", "GarageArea", "GarageCars", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "BsmtFullBath", "BsmtHalfBath",
    "3SsnPorch", "EnclosedPorch", "ScreenPorch", "MasVnrArea", "PoolArea", "YrSold", "YearBuilt", 
] 
cols_fillna_none = [
    "PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond", "GarageCond",
    "BsmtFinType1", "BsmtFinType2", "BsmtExposure", "BsmtCond", "BsmtQual", "MasVnrType", 
] 
cols_fillna_must_ferq = [
    'MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
     'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'YearRemodAdd', 'RoofStyle',
     'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
     'HeatingQC', 'Electrical', 'KitchenAbvGr', 'KitchenQual', 'SaleCondition', 'Functional', 'Fireplaces',
     'PavedDrive', 'SaleType',
] 

In [5]:
df[cols_fillna_zero] = df[cols_fillna_zero].fillna(0)
df[cols_fillna_none] = df[cols_fillna_none].fillna('NA')
df[cols_fillna_must_ferq] = df[cols_fillna_must_ferq].fillna(df.mode().iloc[0])

In [6]:
cols_full_rooms = ['BsmtFullBath', 'FullBath', 'TotRmsAbvGrd']
cols_half_rooms = ['BsmtHalfBath', 'HalfBath']
cols_area_house = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', '3SsnPorch']
cols_area_miscellaneous = ['GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch', 'PoolArea']

In [7]:
df['total_rooms'] = df[cols_full_rooms].sum(axis=1) + (df[cols_half_rooms].sum(axis=1) /2)
df['total_area_house'] = df[cols_area_house].sum(axis=1)
df['total_area_miscellaneous'] = df[cols_area_miscellaneous].sum(axis=1)
df['total_area'] = df[['total_area_house', 'total_area_miscellaneous']].sum(axis=1)
df['percentage_BsmtUnfSF'] = (df['BsmtUnfSF'] / df['TotalBsmtSF']) * 100
df['total_floors'] = df.apply(lambda row: sum(row[c] > 0 for c in cols_area_house[1:]), axis=1)
df['age'] = df['YearBuilt'] - df['YrSold']

In [8]:
cols_to_drop = [
    'Street', 'Utilities', 'Condition2', 'RoofMatl', 'Heating', 'LowQualFinSF', 
    'KitchenAbvGr', '3SsnPorch', 'PoolArea', 'PoolQC', 'MiscFeature', 'MiscVal'
]   # high frequency (near zero variance), great then 0.95
len(cols_to_drop)

12

In [9]:
display(df.shape)
df = df.drop(cols_to_drop, axis=1)
display(df.shape)

(1460, 87)

(1460, 75)

In [10]:
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,...,SaleType,SaleCondition,SalePrice,total_rooms,total_area_house,total_area_miscellaneous,total_area,percentage_BsmtUnfSF,total_floors,age
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65,8450,NA,Reg,Lvl,Inside,Gtl,CollgCr,...,WD,Normal,208500,11.5,2566.0,2319.0,4885.0,17.523364,2,-5
2,20,RL,80,9600,NA,Reg,Lvl,FR2,Gtl,Veenker,...,WD,Normal,181500,8.5,2524.0,2020.0,4544.0,22.503962,1,-31
3,60,RL,68,11250,NA,IR1,Lvl,Inside,Gtl,CollgCr,...,WD,Normal,223500,9.5,2706.0,2436.0,5142.0,47.173913,2,-7
4,70,RL,60,9550,NA,IR1,Lvl,Corner,Gtl,Crawfor,...,WD,Abnorml,140000,9.0,2473.0,2666.0,5139.0,71.428571,2,-91
5,60,RL,84,14260,NA,IR1,Lvl,FR2,Gtl,NoRidge,...,WD,Normal,250000,12.5,3343.0,3310.0,6653.0,42.794760,2,-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62,7917,NA,Reg,Lvl,Inside,Gtl,Gilbert,...,WD,Normal,175000,9.5,2600.0,2147.0,4747.0,100.000000,2,-8
1457,20,RL,85,13175,NA,Reg,Lvl,Inside,Gtl,NWAmes,...,WD,Normal,210000,10.0,3615.0,2922.0,6537.0,38.197147,1,-32
1458,70,RL,66,9042,NA,Reg,Lvl,Inside,Gtl,Crawfor,...,WD,Normal,266500,11.0,3492.0,2652.0,6144.0,76.128472,2,-69


In [11]:
df.to_parquet(f"{data_dir}/interim/train.parq", engine='pyarrow')
# df.to_parquet(f"{data_dir}/interim/test.parq", engine='pyarrow')